In [1]:
import pylab
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import csv
from bs4 import BeautifulSoup  
from wordcloud import WordCloud,STOPWORDS

import nltk
#nltk.download()  # Download text data sets, including stop words
from nltk.corpus import stopwords # Import the stop word list

import operator
import re

print "initial imports complete"

/Applications/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


initial imports complete


In [2]:
#READ IN THE YELP FILES to a pandas data frame
user = pd.read_csv("yelp_academic_dataset_user.csv", index_col = 'user_id')
print "user read-in complete"
print(user.shape)

business = pd.read_csv("yelp_academic_dataset_business.csv", na_filter=False, index_col = 'business_id')
print "business read-in complete"
print(business.shape)

reviews = pd.read_csv("yelp_academic_dataset_review.csv", index_col = 'review_id')
print "reviews read-in complete"
print(reviews.shape)

return_visit = reviews.duplicated(['user_id', 'business_id'])
print 'return_visit array created'

user read-in complete
(552339, 22)
business read-in complete
(77445, 97)
reviews read-in complete
(2225213, 9)
return_visit array created


In [3]:
#Check for users who reviewed the same business multipel times
#"return_visit" marks up the follow-up reviews.
return_visit = reviews.duplicated(['user_id', 'business_id'])
print 'return_visit array created'

reviews['return_visit'] = return_visit
print 'return_visit array added to reviews'
print(reviews.shape)

return_visit array created
return_visit array added to reviews
(2225213, 10)


In [4]:
#first visit marks reviews which were later followed up
first_visit = reviews.duplicated(['user_id', 'business_id'], keep='last')
print 'first_visit array created'

reviews['first_visit'] = first_visit
print 'first_visit array added to reviews'
print(reviews.shape)

first_visit array created
first_visit array added to reviews
(2225213, 11)


In [5]:
#Now pick out just those two sub-sets of reviews (first-visits and return visits)
reviewsR = reviews[reviews.return_visit == True]
print(reviewsR.shape)

reviewsR2 = reviewsR.sort_values(by=['user_id'])
print(reviewsR2.shape)

reviewsF = reviews[reviews.first_visit == True]
print(reviewsF.shape)

reviewsF2 = reviewsF.sort_values(by=['user_id'])
print(reviewsF2.shape)

#For comparison, also pick out reviews with NO return visits
reviewsN = reviews[reviews.return_visit == False]
reviewsN2 = reviewsN[reviewsN.first_visit == False]
print(reviewsN2.shape)

#Pick out GOOD return reviews
reviewsRG = reviewsR[reviewsR.stars == 5]
print(reviewsRG.shape)

#Pick out the BAD return reviews
reviewsRB = reviewsR[reviewsR.stars == 1]
print(reviewsRB.shape)

(69792, 11)
(69792, 11)
(69792, 11)
(69792, 11)
(2093827, 11)
(30214, 11)
(10244, 11)


In [6]:
#Get an id list of the most frequent users (those with 25 or more reviews)
user['user_id2'] = user.index
user_frequent = user[user.review_count > 25]
user_frequent = user_frequent.sort_values(by=['user_id2'])
print(user.shape)
print(user_frequent.shape)

user_frequent_id = user_frequent['user_id2']

id_len = len(user_frequent_id)
print(id_len)

#Select out return reviews written by only the *most frequent* reviewers
reviewsUF = pd.merge(reviewsR2, user_frequent, left_on='user_id', right_on='user_id2')
print(reviewsUF.shape)

reviewsUF = reviewsUF[reviewsUF.stars > 0]
print(reviewsUF.shape)
print(reviewsUF[1:10])


(552339, 23)
(106755, 23)
106755
(45989, 34)
(45989, 34)
                  user_id                                               text  \
1  --65q1FpAL_UQtVZ2PTGew  Just a quick update! \n\nThis place is still a...   
2  --65q1FpAL_UQtVZ2PTGew  I'm am just updating since they are open, and ...   
3  --65q1FpAL_UQtVZ2PTGew  As promised, I returned to Lobby's, tried thei...   
4  --65q1FpAL_UQtVZ2PTGew  I feel like I've been all over with my reviews...   
5  --65q1FpAL_UQtVZ2PTGew  Yet again I'll say it... This is THE Vietnames...   
6  --65q1FpAL_UQtVZ2PTGew  I find myself at CherryBlossom a lot nowadays,...   
7  --VxRvXk3b8FwsSbC2Zpxw  Great place, they do pretty much everything at...   
8  --VxRvXk3b8FwsSbC2Zpxw  Great place! Love the happy hour menu too. The...   
9  --qhwKkTzgBeCH3wEJjg2g  Received a reply to my first review so, we wen...   

   votes.cool_x             business_id  votes.funny_x  stars        date  \
1             2  yVQiGdxmnrkJDyQXv2maNA              2      5  20

In [7]:
#Let's count the stars
starnumR2 = len(reviewsR2.stars)
starnumF2 = len(reviewsF2.stars)
starnumN2 = len(reviewsN2.stars)
starnumUF = len(reviewsUF.stars)
print(starnumR2, starnumF2, starnumN2, starnumUF)

starsR = np.array([0.,0.,0.,0.,0.])  #RETURN review
starsF = np.array([0.,0.,0.,0.,0.])  #FIRST review
starsN = np.array([0.,0.,0.,0.,0.])  #no-return (one-only) review
starsUF = np.array([0.,0.,0.,0.,0.])  #RETURN review of *u*sers who are *f*requent

for i in range(0,5):
    starsR[i] = float(len(reviewsR2[reviewsR2.stars == i+1])) / float(starnumR2) * 100.
    starsF[i] = float(len(reviewsF2[reviewsF2.stars == i+1])) / float(starnumF2) * 100.
    starsN[i] = float(len(reviewsN2[reviewsN2.stars == i+1])) / float(starnumN2) * 100.
    starsUF[i] = float(len(reviewsUF[reviewsUF.stars == i+1])) / float(starnumUF) * 100.
    
print(starsR)
print(starsF)
print(starsN)
print(starsUF)

(69792, 69792, 2093827, 45989)
[ 14.67790005   8.77034617  11.66752636  21.59273269  43.29149473]
[ 12.0601215    8.25452774  12.40256763  26.49587345  40.78690967]
[ 11.5959437    8.53957848  12.71456524  26.73869427  40.41121831]
[ 11.18745787   9.86322816  13.90549914  25.99317228  39.05064254]


In [8]:
#Let's chart the stars
import matplotlib.patches as mpatches

multiple_bars = plt.figure()

x = np.array([1,2,3,4,5])

ax = plt.subplot(111)
ax.bar(x-0.2, starsR,width=0.2,color='b',align='center', label='Return review')
ax.bar(x,     starsF,width=0.2,color='g',align='center', label='Initial review')
ax.bar(x+0.2, starsN,width=0.2,color='r',align='center', label='One-only review')
ax.bar(x+0.4, starsUF,width=0.2,color='k',align='center', label='Frequent-users review')

ax.set_ylabel('Percentage of Reviewers')
ax.set_xlabel('Star Rating out of 5')

blue_patch =mpatches.Patch(color='blue', label='Return review')
green_patch = mpatches.Patch(color='green', label='Intial review')
red_patch = mpatches.Patch(color='red', label='One-only review')
black_patch = mpatches.Patch(color='black', label='Frequent-users review')
ax.legend(handles=[blue_patch, green_patch, red_patch, black_patch], loc=2)

plt.show()
plt.savefig('barchart.png')
plt.clf()
plt.cla()
plt.close()

In [9]:
#datesR = reviewsR2['date']
#datesN = reviewsN2['date']

datesumR = reviewsR2.groupby('date').count()
datesumF = reviewsF2.groupby('date').count()

print(datesumR.tail())
print(datesumF.tail())


            user_id  text  votes.cool  business_id  votes.funny  stars  type  \
date                                                                           
2015-12-20       72    72          72           72           72     72    72   
2015-12-21       68    68          68           68           68     68    68   
2015-12-22       75    75          75           75           75     75    75   
2015-12-23       54    54          54           54           54     54    54   
2015-12-24       43    43          43           43           43     43    43   

            votes.useful  return_visit  first_visit  
date                                                 
2015-12-20            72            72           72  
2015-12-21            68            68           68  
2015-12-22            75            75           75  
2015-12-23            54            54           54  
2015-12-24            43            43           43  
            user_id  text  votes.cool  business_id  votes.fun

In [11]:
num_revR = datesumR.type
num_revF = datesumF.type
print(num_revR[100:110])
print(num_revF[100:110])

plt.clf()
plt.cla()
plt.close()

plt.figure()
plt.subplot(111)

plt.title(' ')
plt.xlabel('Test')
plt.ylabel('Total number of reviews per date')
blue_patch = mpatches.Patch(color='blue', label='Return reviews (70,000 total)')
green_patch = mpatches.Patch(color='green', label='Intial reviews (70,000 total)')
plt.legend(handles=[blue_patch, green_patch], title='Number of reviews per date', loc=3)
plt.plot([0, 1, 2, 3], [0, 1, 2, 3])
plt.show()
plt.savefig("test.png")

num_revR.plot(color='b')
num_revF.plot(color='g')
plt.savefig('n_vs_date.png')

date
2008-08-05    5
2008-08-06    2
2008-08-07    3
2008-08-08    1
2008-08-09    4
2008-08-11    5
2008-08-12    6
2008-08-13    5
2008-08-14    3
2008-08-15    8
Name: type, dtype: int64
date
2007-04-25    1
2007-04-26    1
2007-04-30    1
2007-05-03    1
2007-05-08    1
2007-05-09    1
2007-05-12    2
2007-05-13    1
2007-05-16    1
2007-05-17    1
Name: type, dtype: int64


In [136]:
###NOW FOR A LITTLE NLP

def review_to_words( raw_review ):
	# Cleaning technique learned from Kaggle Bag of Words competition:
    # Function to convert a raw review to a string of words
    #
    #Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    
    #Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    
    #Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
 
    stops = set(stopwords.words("english"))                  

    #Remove stop words
    meaningful_words = [w for w in words if not w in stops]   

    return( " ".join( meaningful_words ))   

In [58]:
#Clean the reviews
reviewsF2_clean=[]
nrev = reviewsF2['text'].size
for i in range( 0, nrev ):
	reviewsF2_clean.append(review_to_words(reviewsF2['text'][i]))
	if(i%1000 == 0):
		print("Cleaned %d of %d so far\n" %(i, nrev))

reviewsR2_clean=[]
nrev = reviewsR2['text'].size
for i in range( 0, nrev ):
	reviewsR2_clean.append(review_to_words(reviewsR2['text'][i]))
	if(i%1000 == 0):
		print("Cleaned %d of %d so far\n" % (i, nrev))
        
#There are more "no-follow-up" reviews, but clean same amount to have data in each set
reviewsN2_clean=[]   
for i in range( 0, nrev ):
	reviewsN2_clean.append(review_to_words(reviewsN2['text'][i]))
	if(i%1000 == 0):
		print("Cleaned %d of %d so far\n" % (i, nrev))

Cleaned 0 of 69792 so far

Cleaned 1000 of 69792 so far

Cleaned 2000 of 69792 so far

Cleaned 3000 of 69792 so far

Cleaned 4000 of 69792 so far

Cleaned 5000 of 69792 so far

Cleaned 6000 of 69792 so far

Cleaned 7000 of 69792 so far

Cleaned 8000 of 69792 so far

Cleaned 9000 of 69792 so far

Cleaned 10000 of 69792 so far

Cleaned 11000 of 69792 so far

Cleaned 12000 of 69792 so far

Cleaned 13000 of 69792 so far

Cleaned 14000 of 69792 so far

Cleaned 15000 of 69792 so far

Cleaned 16000 of 69792 so far

Cleaned 17000 of 69792 so far

Cleaned 18000 of 69792 so far

Cleaned 19000 of 69792 so far

Cleaned 20000 of 69792 so far

Cleaned 21000 of 69792 so far

Cleaned 22000 of 69792 so far

Cleaned 23000 of 69792 so far

Cleaned 24000 of 69792 so far

Cleaned 25000 of 69792 so far

Cleaned 26000 of 69792 so far

Cleaned 27000 of 69792 so far

Cleaned 28000 of 69792 so far

Cleaned 29000 of 69792 so far

Cleaned 30000 of 69792 so far

Cleaned 31000 of 69792 so far

Cleaned 32000 of 6979

In [61]:
#Clean the GOOD and BAD return reviews
reviewsRG_clean=[]
nrev = reviewsRG['text'].size
for i in range( 0, nrev ):
	reviewsRG_clean.append(review_to_words(reviewsRG['text'][i]))
	if(i%1000 == 0):
		print("Cleaned %d of %d so far\n" %(i, nrev))

reviewsRB_clean=[]
nrev = reviewsRB['text'].size
for i in range( 0, nrev ):
	reviewsRB_clean.append(review_to_words(reviewsRB['text'][i]))
	if(i%1000 == 0):
		print("Cleaned %d of %d so far\n" % (i, nrev))

Cleaned 0 of 30214 so far

Cleaned 1000 of 30214 so far

Cleaned 2000 of 30214 so far

Cleaned 3000 of 30214 so far

Cleaned 4000 of 30214 so far

Cleaned 5000 of 30214 so far

Cleaned 6000 of 30214 so far

Cleaned 7000 of 30214 so far

Cleaned 8000 of 30214 so far

Cleaned 9000 of 30214 so far

Cleaned 10000 of 30214 so far

Cleaned 11000 of 30214 so far

Cleaned 12000 of 30214 so far

Cleaned 13000 of 30214 so far

Cleaned 14000 of 30214 so far

Cleaned 15000 of 30214 so far

Cleaned 16000 of 30214 so far

Cleaned 17000 of 30214 so far

Cleaned 18000 of 30214 so far

Cleaned 19000 of 30214 so far

Cleaned 20000 of 30214 so far

Cleaned 21000 of 30214 so far

Cleaned 22000 of 30214 so far

Cleaned 23000 of 30214 so far

Cleaned 24000 of 30214 so far

Cleaned 25000 of 30214 so far

Cleaned 26000 of 30214 so far

Cleaned 27000 of 30214 so far

Cleaned 28000 of 30214 so far

Cleaned 29000 of 30214 so far

Cleaned 30000 of 30214 so far

Cleaned 0 of 10244 so far

Cleaned 1000 of 10244 so 

In [62]:
#Make a couple cute lil' word clouds
mystringN = ''.join(reviewsN2_clean)
mystringF = ''.join(reviewsF2_clean)
mystringR = ''.join(reviewsR2_clean)
mystringRG = ''.join(reviewsRG_clean)
mystringRB = ''.join(reviewsRB_clean)

wordcloudN = WordCloud(max_font_size=40, relative_scaling=.5 , \
                    stopwords=STOPWORDS).generate(mystringN)

wordcloudF = WordCloud(max_font_size=40, relative_scaling=.5 , \
                    stopwords=STOPWORDS).generate(mystringF)

wordcloudR = WordCloud(max_font_size=40, relative_scaling=.5 , \
                    stopwords=STOPWORDS).generate(mystringR)

wordcloudRG = WordCloud(max_font_size=40, relative_scaling=.5 , \
                    stopwords=STOPWORDS).generate(mystringRG)

wordcloudRB = WordCloud(max_font_size=40, relative_scaling=.5 , \
                    stopwords=STOPWORDS).generate(mystringRB)

#general word cloud
plt.figure()

plt.subplot(131)
plt.imshow(wordcloudN)
plt.axis('off')

plt.subplot(132)
plt.imshow(wordcloudF)
plt.axis('off')

plt.subplot(133)
plt.imshow(wordcloudR)
plt.axis('off')

fig = plt.gcf()
fig.set_size_inches(15.5, 6.5)                   
plt.savefig('wordcloud.png')


#good and bad review word cloud
plt.figure()

plt.subplot(121)
plt.imshow(wordcloudRG)
plt.axis('off')

plt.subplot(122)
plt.imshow(wordcloudRB)
plt.axis('off')

fig = plt.gcf()
fig.set_size_inches(15.5, 6.5)                   
plt.savefig('wordcloudGB.png')

In [64]:
# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 500) 


F_features = vectorizer.fit_transform(reviewsF2_clean)

# Numpy arrays are easy to work with, so convert the result to an 
# array
F_features = F_features.toarray()
print ("DONE\n")

print(F_features.shape)

# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
print (vocab)

# Sum up the counts of each vocabulary word
distF = np.sum(F_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, distF):
    print (count, tag)


DONE

(69792, 500)
[u'able', u'absolutely', u'actually', u'add', u'ago', u'almost', u'already', u'also', u'although', u'always', u'amazing', u'amount', u'another', u'anyone', u'anything', u'appetizer', u'appointment', u'area', u'around', u'arrived', u'ask', u'asked', u'ate', u'atmosphere', u'attentive', u'available', u'away', u'awesome', u'back', u'bacon', u'bad', u'bar', u'bbq', u'beans', u'beautiful', u'beef', u'beer', u'believe', u'best', u'better', u'big', u'bill', u'bit', u'bread', u'breakfast', u'bring', u'brought', u'buffet', u'burger', u'burgers', u'business', u'busy', u'buy', u'cake', u'call', u'called', u'came', u'car', u'card', u'care', u'change', u'charge', u'cheap', u'check', u'cheese', u'chef', u'chicken', u'chips', u'chocolate', u'choice', u'clean', u'close', u'club', u'coffee', u'cold', u'come', u'comes', u'comfortable', u'coming', u'company', u'cooked', u'cool', u'could', u'counter', u'couple', u'course', u'crab', u'cream', u'crispy', u'customer', u'customers', u'cut',

In [65]:
print ("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set
forest = forest.fit(F_features, reviewsF2["stars"] )
print("DONE")


Training the random forest...
DONE


In [70]:
# Get a bag of words for the test set, and convert to a numpy array
R_features = vectorizer.transform(reviewsR2_clean)
R_features = R_features.toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(R_features)


In [79]:
# Copy the results to a pandas dataframe with an "id" column and
# a "stars" column

reviewsR2['review_id2'] = reviewsR2.index

reviewsR2_guess = pd.DataFrame( data={"review_id":reviewsR2["review_id2"], 
                                      "stars":reviewsR2["stars"], "stars_guess":result} )

print(reviewsR2_guess.stars[100:110], reviewsR2_guess.stars_guess[100:110])



(review_id
VGYDpYvvq4XI0dR4hZFCUw    4
L5o-vWBEf4B5KbVzmES1LQ    3
QOb794YZrWY8N0a6JLOIUg    3
AxYHV7k0SwaDhK6qjW7O-A    3
00yjEMnxGDwAaVL2J0PSXg    4
NaX_kcjnpy2Z4uIPgW3ySw    4
c0DUCaBnWR9x-NDNE_w2Ww    2
1PP3DRvfokAztQfaYoj9Pw    5
6STQXo2gnsEvFD8Me6A2VQ    5
OSoN590PJPuPl2mejFQ2NA    4
Name: stars, dtype: int64, review_id
VGYDpYvvq4XI0dR4hZFCUw    1
L5o-vWBEf4B5KbVzmES1LQ    1
QOb794YZrWY8N0a6JLOIUg    4
AxYHV7k0SwaDhK6qjW7O-A    1
00yjEMnxGDwAaVL2J0PSXg    4
NaX_kcjnpy2Z4uIPgW3ySw    4
c0DUCaBnWR9x-NDNE_w2Ww    4
1PP3DRvfokAztQfaYoj9Pw    5
6STQXo2gnsEvFD8Me6A2VQ    5
OSoN590PJPuPl2mejFQ2NA    4
Name: stars_guess, dtype: int64)
